In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
city = '대구'

path = 'Data/Variants/SIR/alpha_delta/*.csv'
dic_files = Load_files.load_files(path, -18, -16)
data = dic_files[city]
data

,stdDay,Susceptible,Alpha_Infected,Alpha_Recovered,Alpha_Dead,Alpha_alpha,Alpha_beta,Alpha_gamma,Delta_Infected,Delta_Recovered,Delta_Dead,Delta_alpha,Delta_beta,Delta_gamma
0,2021-04-12,2380065,39.65,163.36,2.00,0.059242,0.058260,0.005296,0.32,0.00,0.00,0.564647,0.000000,0.0
1,2021-04-13,2380056,39.47,165.67,2.21,0.072737,0.079807,0.000000,0.50,0.00,0.00,0.441681,0.000000,0.0
2,2021-04-14,2380045,39.18,168.82,2.21,0.119906,0.037519,0.000000,0.72,0.00,0.00,0.501913,0.000000,0.0
3,2021-04-15,2380027,42.39,170.29,2.21,0.024628,0.084218,0.000000,1.08,0.00,0.00,0.074358,0.000000,0.0
4,2021-04-16,2380023,39.86,173.86,2.21,0.078574,0.052684,0.000000,1.16,0.00,0.00,0.207690,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2021-08-11,2376752,12.67,486.64,3.05,0.030942,0.132597,0.000000,814.69,726.09,0.84,0.045715,0.052928,0.0
122,2021-08-12,2376713,11.38,488.32,3.05,0.050350,0.163445,0.000000,808.62,769.21,0.84,0.067316,0.059039,0.0
123,2021-08-13,2376656,10.09,490.18,3.05,0.081696,0.136769,0.000000,815.03,816.95,0.84,0.096082,0.043459,0.0
124,2021-08-14,2376574,9.53,491.56,3.05,0.060128,0.072403,0.000000,857.51,852.37,0.84,0.063482,0.020653,0.0


In [3]:
path = 'hyperparameter/SIR/alpha/*.pkl'
files = glob.glob(path)
dic_hyperparameter = {}
for file in files:
    dic_hyperparameter[file[25:-4]] = load_hyperparameter(file)
    
dic_hyperparameter

{'BiGRU': [378, 0.001, 50, 8, 64, 32, 0.25],
 'BiLSTM': [431, 0.001, 50, 4, 64, 16, 0.25],
 'BiRNN': [506, 0.0001, 50, 8, 32, 16, 0.25],
 'GRU': [294, 0.001, 50, 4, 64, 32, 0.25],
 'LSTM': [613, 0.001, 50, 4, 32, 32, 0.25],
 'RNN': [434, 0.0001, 50, 8, 32, 32, 0.25],
 'seq2seq_BiGRU': [313,
  0.0001,
  20,
  4,
  64,
  8,
  0.25,
  <function utils.criterion2(actual, predict)>],
 'seq2seq_BiLSTM': [245,
  0.001,
  20,
  4,
  32,
  16,
  0.25,
  <function utils.criterion3(actual, predict)>],
 'seq2seq_BiRNN': [714,
  0.001,
  50,
  4,
  32,
  32,
  0.25,
  <function utils.criterion2(actual, predict)>],
 'seq2seq_GRU': [242,
  0.0001,
  20,
  1,
  32,
  8,
  0.25,
  <function utils.criterion2(actual, predict)>],
 'seq2seq_LSTM': [526, 0.001, 20, 2, 32, 32, 0.25, MSELoss()],
 'seq2seq_RNN': [331,
  0.001,
  50,
  1,
  32,
  32,
  0.25,
  <function utils.criterion2(actual, predict)>]}

In [4]:
criterion = nn.MSELoss()

input_size = 3
sequence_length = 60
num_epochs = 10000

df = Prepare_df.processing(data, 'stdDay', 'Alpha_alpha')

x = df.iloc[:, 0:]
y = df.iloc[:,:1]

ms = MinMaxScaler()
ss = StandardScaler()

ss.fit(x)
ms.fit(y)

MinMaxScaler()

In [5]:
trained_cities = {}
trained_models = {}
city_list = ['대구', '세종', '광주', '경기', '경남']
models_list = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU', 
               'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']


for city in tqdm(city_list, desc = 'cities', position = 0):
    data = dic_files[city]
    
    input_size = 3
    sequence_length = 60
    num_epochs = 10000

    df = Prepare_df.processing(data, 'stdDay', 'Alpha_alpha')

    x = df.iloc[:, 0:]
    y = df.iloc[:,:1]

    ms = MinMaxScaler()
    ss = StandardScaler()

    ss.fit(x)
    ms.fit(y)

    for num_model in tqdm(range(12), desc = f'{city}', position = 1):
        rmse_min = np.inf
        model_name = models_list[num_model]
        lr = dic_hyperparameter[model_name][1]
        patience = dic_hyperparameter[model_name][2]
        num_layers = dic_hyperparameter[model_name][3]
        batch_size = dic_hyperparameter[model_name][4]
        hidden_size = dic_hyperparameter[model_name][5]
        dropout = dic_hyperparameter[model_name][6]

        if num_model < 6:
            criterion = nn.MSELoss()
        else:
            criterion = dic_hyperparameter[model_name][7]
        #print('-------------------------------------------------------------------------------------')
        #print('lr:', lr, ' patience:', patience, ' num_layers:', num_layers, 
        #      ' batch_size:', batch_size, ' hidden_size:', hidden_size, ' dropout:', dropout)

        if num_model < 6:
            x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 50, 60, 1, batch_size, 'mto')
        else:
            x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 37, 60, 14, batch_size, 'mtm')

        if num_model == 0:
            model = RNN(input_size = input_size,
                        hidden_size = hidden_size,
                        sequence_length = sequence_length,
                        num_layers = num_layers, 
                        dropout = dropout, 
                        device = device).to(device)

        elif num_model == 1:
            model = LSTM(input_size = input_size,
                         hidden_size = hidden_size,
                         sequence_length = sequence_length,
                         num_layers = num_layers, 
                         dropout = dropout, 
                         device = device).to(device)

        elif num_model == 2:
            model = GRU(input_size = input_size,
                        hidden_size = hidden_size,
                        sequence_length = sequence_length,
                        num_layers = num_layers, 
                        dropout = dropout, 
                        device = device).to(device)

        elif num_model == 3:
            model = BiRNN(input_size = input_size,
                          hidden_size = hidden_size,
                          sequence_length = sequence_length,
                          num_layers = num_layers, 
                          dropout = dropout, 
                          device = device).to(device)

        elif num_model == 4:
            model = BiLSTM(input_size = input_size,
                           hidden_size = hidden_size,
                           sequence_length = sequence_length,
                           num_layers = num_layers, 
                           dropout = dropout, 
                           device = device).to(device)

        elif num_model == 5:
            model = BiGRU(input_size = input_size,
                          hidden_size = hidden_size,
                          sequence_length = sequence_length,
                          num_layers = num_layers, 
                          dropout = dropout, 
                          device = device).to(device)

        elif num_model == 6:
            model = RNN_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)

        elif num_model == 7:
            model = LSTM_encoder_decoder(input_size = input_size, 
                                         hidden_size = hidden_size,
                                         num_layers = num_layers, 
                                         dropout = dropout,
                                         device = device).to(device)

        elif num_model == 8:
            model = GRU_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)

        elif num_model == 9:
            model = BiRNN_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

        elif num_model == 10:
            model = BiLSTM_encoder_decoder(input_size = input_size, 
                                              hidden_size = hidden_size,
                                              num_layers = num_layers, 
                                              dropout = dropout,
                                              device = device).to(device)

        elif num_model == 11:
            model = BiGRU_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)            

        optimizer = Adam(model.parameters(), lr = lr)

        if num_model < 6:
            loss_list, model, epoch = Trainer.Many_to_One(train_loader, 
                                                          test_loader, 
                                                          model, 
                                                          criterion, 
                                                          optimizer, 
                                                          num_epochs, 
                                                          patience,
                                                          device)

        else:
            loss_list, model, epoch = Trainer.Many_to_Many(train_loader, 
                                                           test_loader, 
                                                           model, 
                                                           criterion, 
                                                           optimizer, 
                                                           num_epochs, 
                                                           patience,
                                                           14,
                                                           device)            

        trained_models[models_list[num_model]] = model
        
                
    trained_cities[city] = trained_models

cities:   0%|          | 0/5 [00:00<?, ?it/s]

대구:   0%|          | 0/12 [00:00<?, ?it/s]

세종:   0%|          | 0/12 [00:00<?, ?it/s]

광주:   0%|          | 0/12 [00:00<?, ?it/s]

경기:   0%|          | 0/12 [00:00<?, ?it/s]

경남:   0%|          | 0/12 [00:00<?, ?it/s]

In [6]:
for city in trained_cities:
    for name, model in trained_cities[city].items():
        save_model(model.state_dict(), f"model/SIR/{city}/alpha/{name}.pth")